In [8]:
BASELINE_TO_SAVE = "uniform" # "mean" or "uniform". Decides which baseline is used to create the submission.csv
DATA_PREPARATION_VOTE_METHOD = "sum_and_normalize" # "max_vote_window" or "sum_and_normalize". Decides how to aggregate the predictions of the overlapping windows

In [9]:
import os
from tqdm.auto import tqdm
import sys
if bool(os.environ.get("KAGGLE_URL_BASE", "")):
  import sys
  # running on kaggle
  sys.path.insert(0, "/kaggle/input/hsm-source-files")
else:
  # running locally
  sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))

import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


from src.utils.utils import get_raw_data_dir, get_processed_data_dir, get_submission_csv_path, running_in_kaggle
from src.utils.constants import Constants
from src.datasets.eeg_processor import EEGDataProcessor

from src.datasets.spectrogram_dataset import SpectrogramDataset
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path
from src.models.base_cnn import BaseCNN
import timm

In [10]:
DATA_PATH = get_raw_data_dir()
PROCESSED_TRAIN_DATA_PATH = get_processed_data_dir() / "train_processed.csv"

if (running_in_kaggle()):
  # preprocess data only if running in kaggle, locally it's already done
  processor = EEGDataProcessor(raw_data_path=DATA_PATH, processed_data_path=get_processed_data_dir())
  train_df = processor.process_data(vote_method=DATA_PREPARATION_VOTE_METHOD, skip_npy=True)
else:
  # load preprocessed data
  train_df = pd.read_csv(PROCESSED_TRAIN_DATA_PATH)

test_df = pd.read_csv(DATA_PATH / "test.csv")

kl_score = nn.KLDivLoss(reduction="batchmean")

In [11]:
train_df.head()

,eeg_id,spectrogram_id,min_offset,max_offset,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,789577333,0.0,16.0,20654,Other,0.0,0.000000,0.25,0.000000,0.166667,0.583333
1,582999,1552638400,0.0,38.0,20230,LPD,0.0,0.857143,0.00,0.071429,0.000000,0.071429
2,642382,14960202,1008.0,1032.0,5955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000
3,751790,618728447,908.0,908.0,38549,GPD,0.0,0.000000,1.00,0.000000,0.000000,0.000000
4,778705,52296320,0.0,0.0,40955,Other,0.0,0.000000,0.00,0.000000,0.000000,1.000000


In [12]:
test_df.head()

,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [ ]:
class InfCFG:
    """Configuration for inference."""
    data_path = DATA_PATH
    model_name = 'tf_efficientnet_b0_ns'
    target_size = 6
    img_size = (128, 256)
    n_splits = 5
    batch_size = 64
    num_workers = 0
    MODEL_DIR = '/kaggle/input/basecnnfolds/' if 'KAGGLE_URL_BASE' in os.environ else '../../../notebooks/'

InfCFG.model_paths = [os.path.join(InfCFG.MODEL_DIR, f'best_model_fold{i}.pth') for i in range(InfCFG.n_splits)]


In [14]:
def run_inference():
    """Executes the main inference loop."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    test_df = pd.read_csv(os.path.join(InfCFG.data_path, 'test.csv'))
    test_dataset = SpectrogramDataset(
        df=test_df, 
        targets=Constants.TARGETS, 
        data_path=InfCFG.data_path, 
        img_size=InfCFG.img_size, 
        mode='test'
    )
    test_loader = DataLoader(
        test_dataset, batch_size=InfCFG.batch_size, shuffle=False, num_workers=InfCFG.num_workers
    )

    all_fold_predictions = []

    for i, path in enumerate(InfCFG.model_paths):
        print(f"\n========== Inferencing with Fold {i} Model ==========")
        if not os.path.exists(path):
            print(f"Model file not found: {path}. Skipping this fold.")
            continue
            
        model = BaseCNN(InfCFG.model_name, pretrained=False, num_classes=InfCFG.target_size)
        model.load_state_dict(torch.load(path, map_location=device))
        model.to(device)
        model.eval()

        current_fold_preds = []
        with torch.no_grad():
            for images in tqdm(test_loader, desc=f"Predicting Fold {i}"):
                outputs = model(images.to(device))
                probs = F.softmax(outputs, dim=1).cpu().numpy()
                current_fold_preds.append(probs)
        
        all_fold_predictions.append(np.concatenate(current_fold_preds))

    if not all_fold_predictions:
        print("No models were found for inference. Aborting.")
        return

    avg_predictions = np.mean(all_fold_predictions, axis=0)
    submission = pd.DataFrame({"eeg_id": test_df["eeg_id"]})
    submission[Constants.TARGETS] = avg_predictions

    submission.to_csv(get_submission_csv_path(), index=False)
    
    print("\nEnsemble predictions: ", avg_predictions)


if __name__ == '__main__':
    run_inference()

Using device: cuda

========== Inferencing with Fold 0 Model ==========


/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


Predicting Fold 0:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 1 Model ==========


Predicting Fold 1:   0%|          | 0/1 [00:00<?, ?it/s]

...


========== Inferencing with Fold 3 Model ==========


Predicting Fold 3:   0%|          | 0/1 [00:00<?, ?it/s]


========== Inferencing with Fold 4 Model ==========


Predicting Fold 4:   0%|          | 0/1 [00:00<?, ?it/s]


Ensemble predictions:  [[0.30127007 0.12761357 0.00676326 0.20392604 0.02217557 0.3382515 ]]
